In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from datetime import datetime
from datetime import timedelta
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import support.ts_class as ts_class
import support.load_and_process_data as lpdata
#import yfinance as yf
import investpy
import math
#import plotly.graph_objects as go
import gspread
from time import sleep

In [2]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/anaconda3/envs/traiding_spark_delta/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/esak/.ivy2/cache
The jars for the packages stored in: /Users/esak/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-edd7ddf3-4022-469b-9c2d-b9ab3de79001;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 278ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-core_2.12;1.2.1 from central in [default]
	io.delta#delta-storage;1.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evic

In [3]:
from delta.tables import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.pandas as ps

In [4]:
import nasdaqdatalink

In [5]:
nasdaqdatalink.read_key(filename="~/.nasdaq/data_link_apikey")

In [6]:
nasdaq_path = "/Users/esak/Projects/stock_traiding/forecast_indices/datasets/nasdaq"

In [9]:
#metadata_files = ['source_data/UMICH_metadata.csv','source_data/USTREASURY_metadata.csv']
metadata_files = ['source_data/USTREASURY_metadata.csv']

In [10]:
calls = 0
if calls == 20:
    print("sleeping 10 minutes...")
    sleep(60*10) #sleep for 10minutes
    calls = 0
    
for f in metadata_files:
    metadata = pd.read_csv(f)
    data_name=f.split("_")[1].split("/")[1]
    for code in metadata.loc[5:,"code"]:
        data_path = "{0}/{1}".format(data_name,code)
        data = nasdaqdatalink.get(data_path)
        data = data.astype(float)
        data.index.rename("Datetime",inplace = True)
        data = data.reset_index()
        data.columns = data.columns.str.replace(";","")
        data.columns = data.columns.str.replace(" ","")
        data.columns = data.columns.str.replace(",",".")
        data.columns = data.columns.str.replace("{","[")
        data.columns = data.columns.str.replace("}","]")
        data.columns = data.columns.str.replace(")","]")
        data.columns = data.columns.str.replace("(","[")
        if code == 'MATDIS':
            data.columns = ['Datetime','TotalOutstanding[bln]','1Year','1Year_%','1-5Years','1-5Years_%','5-10Years','5-10Years_%','10-20Years','%_10-20Years','20+Years','%_20+Years','AvgLength[months]']
        
        data = ps.from_pandas(data).to_spark()

        fn = nasdaq_path+"/"+data_path
        
        print("writing {0}".format(fn))
        data.write.format("delta").mode("overwrite").save(fn)
        calls += 1
    

/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace("{","[")
/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace("}","]")
/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace(")","]"

writing /Users/esak/Projects/stock_traiding/forecast_indices/datasets/nasdaq/USTREASURY/MATDIS


22/10/27 16:05:53 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace("{","[")
/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace("}","]")
/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In

writing /Users/esak/Projects/stock_traiding/forecast_indices/datasets/nasdaq/USTREASURY/MKTDM


/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace("{","[")
/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace("}","]")
/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace(")","]"

...

writing /Users/esak/Projects/stock_traiding/forecast_indices/datasets/nasdaq/USTREASURY/TMBOR


/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace("{","[")
/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace("}","]")
/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace(")","]"

writing /Users/esak/Projects/stock_traiding/forecast_indices/datasets/nasdaq/USTREASURY/TNMBOR


/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace("{","[")
/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace("}","]")
/var/folders/t4/btbs8b8d5c3530kqrnyv6z1h0000gn/T/ipykernel_16567/30869090.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace(")","]"

writing /Users/esak/Projects/stock_traiding/forecast_indices/datasets/nasdaq/USTREASURY/YIELD


In [ ]:
data.columns = ['TotalOutstanding[bln]','1Year','1Year_%','1-5Years','1-5Years_%','5-10Years','5-10Years_%','10-20Years','%_10-20Years','20+Years','%_20+Years','AvgLength[months]']

In [ ]:
        data = ps.from_pandas(data).to_spark()

        fn = nasdaq_path+"/"+data_path
        
        print("writing {0}".format(fn))
        data.write.format("delta").mode("overwrite").save(fn)

In [ ]:
data.write.format("delta").mode("overwrite").save(fn)

In [ ]:
metadata.columns = ['code_test', 'name', 'description', 'refreshed_at', 'from_date', 'to_date']

In [ ]:
metadata.columns.str.strip("_")

In [ ]:
metadata.columns = metadata.columns.str.replace("_","")

In [ ]:
metadata

In [11]:
data.show()

+-------------------+----+----+----+----+----+----+----+----+----+----+----+----+
|           Datetime| 1MO| 2MO| 3MO| 6MO| 1YR| 2YR| 3YR| 5YR| 7YR|10YR|20YR|30YR|
+-------------------+----+----+----+----+----+----+----+----+----+----+----+----+
|1990-01-02 00:00:00|null|null|7.83|7.89|7.81|7.87| 7.9|7.87|7.98|7.94|null| 8.0|
|1990-01-03 00:00:00|null|null|7.89|7.94|7.85|7.94|7.96|7.92|8.04|7.99|null|8.04|
|1990-01-04 00:00:00|null|null|7.84| 7.9|7.82|7.92|7.93|7.91|8.02|7.98|null|8.04|
|1990-01-05 00:00:00|null|null|7.79|7.85|7.79| 7.9|7.94|7.92|8.03|7.99|null|8.06|
|1990-01-08 00:00:00|null|null|7.79|7.88|7.81| 7.9|7.95|7.92|8.05|8.02|null|8.09|
|1990-01-09 00:00:00|null|null| 7.8|7.82|7.78|7.91|7.94|7.92|8.05|8.02|null| 8.1|
|1990-01-10 00:00:00|null|null|7.75|7.78|7.77|7.91|7.95|7.92| 8.0|8.03|null|8.11|
|1990-01-11 00:00:00|null|null| 7.8| 7.8|7.77|7.91|7.95|7.94|8.01|8.04|null|8.11|
|1990-01-12 00:00:00|null|null|7.74|7.81|7.76|7.93|7.98|7.99|8.07| 8.1|null|8.17|
|1990-01-16 00:0